<a href="https://colab.research.google.com/github/surajjorwar/Book-Recommendation-System/blob/main/Book_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [57]:
#installing fuzzywuzzy library
!pip install fuzzywuzzy

In [58]:
#importing library's
import numpy as np
import pandas as pd
from fuzzywuzzy import fuzz, process
from IPython.display import display, Image, HTML

In [59]:
#loading csv files
book = pd.read_csv('/content/drive/MyDrive/BOOK_RECOMMENDER_SYSTEM/Copy of Books.csv')
rating = pd.read_csv('/content/drive/MyDrive/BOOK_RECOMMENDER_SYSTEM/Copy of Ratings.csv')
users = pd.read_csv('/content/drive/MyDrive/BOOK_RECOMMENDER_SYSTEM/Copy of Users.csv')

<ipython-input-59-b36acff5a32a>:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  book = pd.read_csv('/content/drive/MyDrive/BOOK_RECOMMENDER_SYSTEM/Copy of Books.csv')


# **EXPLORING THE DATA **

In [60]:
book.head(2)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...


In [61]:
book.shape

(271360, 8)

In [62]:
book.isna().sum()

ISBN                   0
Book-Title             0
Book-Author            1
Year-Of-Publication    0
Publisher              2
Image-URL-S            0
Image-URL-M            0
Image-URL-L            3
dtype: int64

In [63]:
#droping null values
book.dropna(inplace = True)

In [64]:
book.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 271354 entries, 0 to 271359
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271354 non-null  object
 1   Book-Title           271354 non-null  object
 2   Book-Author          271354 non-null  object
 3   Year-Of-Publication  271354 non-null  object
 4   Publisher            271354 non-null  object
 5   Image-URL-S          271354 non-null  object
 6   Image-URL-M          271354 non-null  object
 7   Image-URL-L          271354 non-null  object
dtypes: object(8)
memory usage: 18.6+ MB


In [65]:
# droping duplicate values if any
book.drop_duplicates(inplace = True )
book.shape

(271354, 8)

In [66]:
#exploring rating dataframe
rating.head(2)

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5


In [67]:
rating.shape

(1149780, 3)

In [68]:
#checking null values
rating.isnull().sum()

User-ID        0
ISBN           0
Book-Rating    0
dtype: int64

In [69]:
# droping duplicate values if any
rating.drop_duplicates(inplace = True )
rating.shape


(1149780, 3)

In [70]:
#Exploring user data frame
users.head(2)

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0


In [71]:
# checing the size of data frame
users.shape

(278858, 3)

In [72]:
#checking null values
users.isnull().sum()

User-ID          0
Location         0
Age         110762
dtype: int64

In [73]:
#filling null values of age column by its mean value
users['Age'].fillna(users['Age'].mean(),inplace = True)

In [74]:
# droping duplicate values if any
users.drop_duplicates(inplace = True )
users.shape

(278858, 3)

In [75]:
# renaming column names   for book dataframe
book.rename(columns = {'Book-Title' : 'title', 'Book-Author' : 'auther', 'Year-Of-Publication' : 'year','Publisher':'publisher','Image-URL-L':'image_url_l' },inplace = True)


In [76]:
# renaming column names for user dataframe
users.rename(columns = {'User-ID' : 'user_id', 'Location' : 'location', 'Age' : 'age'} ,inplace = True)

In [77]:
# renaming column names for rating dataframe
rating.rename(columns = {'User-ID' : 'user_id', 'Book-Rating': 'rating' } ,inplace = True)

In [78]:
# taking only required columns for book dataframe ----->
book = book[['ISBN', 'title', 'auther', 'year', 'publisher','image_url_l']]
book.head(2)

,ISBN,title,auther,year,publisher,image_url_l
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...


In [79]:
#final shape of our data frames
print('shape of book dataframe ', book.shape)
print('shape of users dataframe ',users.shape)
print('shape of rating dataframe ',rating.shape)

shape of book dataframe  (271354, 6)
shape of users dataframe  (278858, 3)
shape of rating dataframe  (1149780, 3)


In [80]:
#to make our data more valuable we will apply some conditions
# taking the users who rated  more than 200 books
x = rating['user_id'].value_counts()>200

In [81]:
#it returns  user_id's
y = x[x].index

In [82]:
#we are filtering the data by these user id  stored in y whe are takking only thpse uise ids  which match with  user ids in y
rating= rating[rating['user_id'].isin(y)]



In [83]:
rating.shape

(526356, 3)

In [84]:
#we are merging books and rating data frames  by using ISBN AS common column
books_with_rating = rating.merge(book , on = 'ISBN')

In [85]:
books_with_rating.shape

(487666, 8)

In [86]:
# no of rating  by title
no_of_rating = books_with_rating.groupby('title')['rating'].count().reset_index()
no_of_rating.rename(columns = {'rating' : 'no_of_rating'},inplace = True )


In [87]:
no_of_rating.shape

(160265, 2)

In [88]:
#we are merging books_with_rating and no_of_rating data frames  by using title AS common column
final_ratings = books_with_rating.merge(no_of_rating ,on = 'title')

In [89]:
final_ratings.head()

,user_id,ISBN,rating,title,auther,year,publisher,image_url_l,no_of_rating
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
3,12538,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
4,13552,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82


In [90]:
final_ratings.drop_duplicates(['user_id','title'],inplace= True)
final_ratings.shape

(483418, 9)

In [91]:
# we are talking only those books whose rating is more than 50
final_ratings = final_ratings[final_ratings['no_of_rating'] >= 50]
final_ratings.shape

(59850, 9)

In [92]:
#creating title and url data frame to shoe image
title_url = final_ratings[['title','image_url_l']]
title_url.shape

(59850, 2)

In [93]:
#we are using pivot table to convert each user id as a columns,title as index and   rating  as rows
pivot = final_ratings.pivot_table(columns = 'user_id' ,index =  'title', values = 'rating')
pivot.shape

(742, 888)

In [94]:
pivot.head()

user_id,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
1st to Die: A Novel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2nd Chance,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN
4 Blondes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84 Charing Cross Road,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN


In [95]:
pivot.fillna(0,inplace =True )

In [96]:
pivot.head(2)

user_id,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [97]:
#we are using sparse matrix because our pivot  contains large no of zeros
from scipy.sparse import csr_matrix
sparse_book = csr_matrix(pivot)

In [98]:
sparse_book

<742x888 sparse matrix of type '<class 'numpy.float64'>'
	with 14942 stored elements in Compressed Sparse Row format>

In [99]:
#importing nearest neighbors
from sklearn.neighbors import NearestNeighbors
nn = NearestNeighbors(algorithm= 'brute')

In [100]:
#training the model
nn.fit(sparse_book)

NearestNeighbors(algorithm='brute')

we have cerated a function book_recomend which  first takes input string of book the major problem was we have to put exatct word to word title name without any error to over come this we use fuzzywuzzy library  
which  compares input string  with existing title names , and filter the result with the title matching  more than 80 percent with input string .
after that we fit our model  and get recommendations ,
at the end we map the  recomended title with image url and with the help of ipython.display library we convert url to images


In [109]:
sparse_book

<742x888 sparse matrix of type '<class 'numpy.float64'>'
	with 14942 stored elements in Compressed Sparse Row format>

In [110]:
#functing
def book_recomend():
  partial_input =input("Please enter a book name: ")
  b = pivot.iloc[:,1].reset_index()
  #list of books
  l = b['title'].tolist()
  new_title_url = title_url[title_url['title'].isin(l)]
  new_title_url.drop_duplicates(['title'],inplace = True)
  # Perform fuzzy matching
  matches = process.extract(partial_input, l, limit=1)
  # Filter matches above a certain similarity threshold
  threshold = 80  # Adjust as needed
  filtered_matches = [match for match in matches if match[1] >= threshold]

  for match in filtered_matches:
    book_name = match[0]

    book_id =  np.where(pivot.index == book_name)[0][0]
    distance ,suggestions  = nn.kneighbors(sparse_book[book_id].reshape(1,-1), n_neighbors=6)
    print('its ' ,book_name, ' book')
    #print(suggestions)
    for i in range(len(suggestions)):
      x= pivot.index[suggestions[i]]

    for i in x[1:] :
      #print(i)
      list_b = []
      list_b.append(i)
      newdf =new_title_url[new_title_url['title'].isin(list_b)]
      for index, row in newdf.iterrows():
        title = row['title']
        image_url = row['image_url_l']
        display(HTML(f"<h2>{title}</h2>"))
        display(HTML(f'<img src="{image_url}" width="300"/>'))



In [111]:
book_recomend()

Please enter a book name: love
its  Beloved  book
